# Example 11-1. Word Embeddings with GloVe and Sentiment Analysis 


- GloVe
    - window 안에 속하는 단어들만을 반영하는 word2vec의 단점을 해결하기 위한 아이디어
    - 전체 dictionary에서 두 단어의 동시등장(co-occurrence)하는 확률을 계산하고 동시에 등장하는 확률이 높을 수록 두 단어 벡터가 가까워지도록 학습
    

## Toy example

In [8]:
#!pip install glove_python-binary

In [9]:
input_text = [['the', 'da', 'vinci', 'code', 'book', 'is', 'just', 'awesome', '.'],
              ['i', 'liked', 'the', 'da', 'vinci', 'code', 'a', 'lot', '.']]

- 입력 데이터는 각 문장을 단어들의 list로 표현하여 준비

In [10]:
input_text

[['the', 'da', 'vinci', 'code', 'book', 'is', 'just', 'awesome', '.'],
 ['i', 'liked', 'the', 'da', 'vinci', 'code', 'a', 'lot', '.']]

In [11]:
from glove import Corpus, Glove
corpus = Corpus() 
corpus.fit(input_text, window=10)

- Co-occurrence를 계산

In [12]:
corpus.dictionary

{'the': 0,
 'da': 1,
 'vinci': 2,
 'code': 3,
 'book': 4,
 'is': 5,
 'just': 6,
 'awesome': 7,
 '.': 8,
 'i': 9,
 'liked': 10,
 'a': 11,
 'lot': 12}

In [13]:
print(corpus.matrix)

  (0, 1)	2.0
  (0, 2)	0.5
  (0, 2)	0.5
  (0, 3)	0.6666666865348816
  (0, 4)	0.25
  (0, 5)	0.20000000298023224
  (0, 6)	0.1666666716337204
  (0, 7)	0.1428571492433548
  (0, 8)	0.2916666865348816
  (0, 9)	0.5
  (0, 10)	1.0
  (0, 11)	0.25
  (0, 12)	0.20000000298023224
  (1, 2)	1.0
  (1, 2)	1.0
  (1, 3)	1.0
  (1, 4)	0.3333333432674408
  (1, 5)	0.25
  (1, 6)	0.20000000298023224
  (1, 7)	0.1666666716337204
  (1, 8)	0.34285715222358704
  (1, 9)	0.3333333432674408
  (1, 10)	0.5
  (1, 11)	0.3333333432674408
  (1, 12)	0.25
  :	:
  (3, 8)	0.5333333611488342
  (3, 9)	0.20000000298023224
  (3, 10)	0.25
  (3, 11)	1.0
  (3, 12)	0.5
  (4, 5)	1.0
  (4, 6)	0.5
  (4, 7)	0.3333333432674408
  (4, 8)	0.25
  (5, 6)	1.0
  (5, 7)	0.5
  (5, 8)	0.3333333432674408
  (6, 7)	1.0
  (6, 8)	0.5
  (7, 8)	1.0
  (8, 9)	0.125
  (8, 10)	0.1428571492433548
  (8, 11)	0.5
  (8, 12)	1.0
  (9, 10)	1.0
  (9, 11)	0.1666666716337204
  (9, 12)	0.1428571492433548
  (10, 11)	0.20000000298023224
  (10, 12)	0.1666666716337204
  (11, 12

In [14]:
glove = Glove(no_components=5)
glove.fit(corpus.matrix, epochs=30)

- 위에서 생성한 co-occurrence matrix를 입력값으로 받아 glove 학습

In [15]:
glove.add_dictionary(corpus.dictionary)

In [16]:
glove.word_vectors # embedding matrix

array([[ 0.04568983, -0.10630906, -0.06833508, -0.00521731,  0.02551073],
       [-0.02210793, -0.07019449, -0.00827241, -0.01887154, -0.06760364],
       [-0.07122593, -0.03427791, -0.08239786, -0.02330353, -0.02362877],
       [-0.00138935,  0.09314505,  0.05422496, -0.02673546,  0.0828445 ],
       [-0.10027092,  0.0950792 , -0.09305676,  0.04907259,  0.02864791],
       [ 0.07688888,  0.09343062, -0.07319521, -0.05993485, -0.07205006],
       [ 0.04726897,  0.00566121,  0.08023728,  0.07804979, -0.01328752],
       [ 0.05259584, -0.04637605,  0.07081554,  0.05173817, -0.03134926],
       [ 0.00664143, -0.08133186,  0.10216001, -0.02002165,  0.06465788],
       [ 0.09795657,  0.02245815,  0.0151073 , -0.08012637, -0.04895071],
       [ 0.0024732 , -0.04678153, -0.02913527,  0.09285719, -0.06221956],
       [-0.02355928,  0.02773438, -0.00809839,  0.04283896, -0.03434178],
       [-0.02863654,  0.06917519, -0.00346973, -0.07914805, -0.05833579]])

## Sentiment Analysis

#### Data preprocessing


- 문서를 line별로 읽어들이면서 단어의 빈도 계산

In [17]:

MAX_FEATURES = 2000   
MAX_SENTENCE_LENGTH = 40  

import collections
import os 
import numpy as np
import nltk
nltk.download('punkt')

maxlen = 0
word_freqs = collections.Counter()
num_recs = 0
ftrain = open("data/umich-sentiment-train.txt", 'rb')
for line in ftrain:
    label, sentence = line.decode('utf8').strip().split("\t")
    words = nltk.word_tokenize(sentence.lower())
    if len(words) > maxlen:
        maxlen = len(words)  # the maximum number of words in a sentence
    for word in words:
        word_freqs[word] += 1  # frequency for each word
    num_recs += 1 # total number of records
ftrain.close()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\TJ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


- 등장 빈도를 기준으로 `MAX_FEATURES` 만큼의 단어를 vocabulary로 결정
- vocabulary에 속하지 않는 단어는 "UNK"로 표시하면서 문장을 단어 단위로 tokenize 하고 list로 저장

In [18]:
vocab = [v for v, _ in word_freqs.most_common(MAX_FEATURES)]

sentences = np.empty((num_recs, ), dtype=list)
i = 0
ftrain = open("data/umich-sentiment-train.txt", 'rb')

for line in ftrain:
    label, sentence = line.decode('utf8').strip().split("\t")
    words = nltk.word_tokenize(sentence.lower())
    sentence = []
    for word in words:
        if word in vocab:
            sentence.append(word)
        else:
            sentence.append("UNK")
    sentences[i] = sentence
    i += 1
    
ftrain.close()

sentences=list(sentences)

In [19]:
len(sentences)

7086

In [20]:
sentences[0]

['the', 'da', 'vinci', 'code', 'book', 'is', 'just', 'awesome', '.']

<font color=blue>

TO DO: GloVe 알고리즘을 통해 word embedding을 하고 embedding matrix를 `embedding_matrix_glove`의 이름으로 저장하시오. 

In [21]:
EMBEDDING_SIZE = 128
corpus = Corpus() 
corpus.fit(sentences, window=10)
glove = Glove(no_components=EMBEDDING_SIZE)
glove.fit(corpus.matrix, epochs=30)
glove.add_dictionary(corpus.dictionary)

In [22]:
embedding_matrix_glove = glove.word_vectors

In [23]:
embedding_matrix_glove.shape

(2001, 128)

<font color=blue>

TO DO: Word2vec 알고리즘을 사용하는 Lecture 11의 예제와 동일하게 이후 과정 진행

- Embedding matrix에 "UNK"을 나타내는 0 행을 추가 
- Look-up dictionary 생성


In [24]:
embedding_matrix_glove = np.append(np.zeros((1,EMBEDDING_SIZE)), embedding_matrix_glove, axis=0)

index2word = {i+1: w for i, w in enumerate(glove.dictionary)} 
index2word[0] = 'PAD'
word2index = {w: i for i, w in index2word.items() }

vocab_size = len(index2word)

- Keras embedding layer에 입력하기 위해 단어 인덱스를 사용하여 문장을 list로 변환하여 저장하고 각 문장의 sentiment label 저장

In [26]:
from keras.preprocessing import sequence


X = np.empty((num_recs, ), dtype=list)
y = np.zeros((num_recs, ))
i = 0
ftrain = open("data/umich-sentiment-train.txt", 'rb')

for line in ftrain:
    label, sentence = line.decode('utf8').strip().split("\t")
    words = nltk.word_tokenize(sentence.lower())
    seqs = []
    for word in words:
        if word in word2index:
            seqs.append(word2index[word])
        else:
            seqs.append(word2index["UNK"])
    X[i] = seqs
    y[i] = int(label)
    i += 1
ftrain.close()
X = sequence.pad_sequences(X, maxlen=MAX_SENTENCE_LENGTH)

In [27]:
X[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [28]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=0)
print(Xtrain.shape, Xtest.shape, ytrain.shape, ytest.shape)

(5668, 40) (1418, 40) (5668,) (1418,)


<font color=blue>

TO DO: 

GloVe에 의해 학습된 embedding matrix를 사용하여 word2vec를 사용해 학습했던 모형과 동일한 구조의 모형을 학습하고 test set에 대한 accuracy를 계산하시오.

In [29]:
BATCH_SIZE = 512
NUM_EPOCHS = 100
EMBEDDING_SIZE = 128

from keras.models import Sequential
from keras.layers import Embedding, Dropout, LSTM, Dense
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_SIZE, input_length = MAX_SENTENCE_LENGTH, mask_zero = True,
                    weights = [embedding_matrix_glove], trainable = False))
model.add(LSTM(32, recurrent_dropout = 0.2, return_sequences = False))
model.add(Dense(1, activation = "sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["acc"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 128)           256256    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                20608     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 276,897
Trainable params: 20,641
Non-trainable params: 256,256
_________________________________________________________________


In [31]:
import time
now = time.strftime("%c")
callbacks_list = [
    ModelCheckpoint(filepath='models/sentiment_analysis_glove.h5', monitor='val_loss', save_best_only=True),
    TensorBoard(log_dir='logs/sentiment_analysis_glove/', histogram_freq=1, embeddings_freq=1,
                # histogram_freq=1 : 1 에포크마다 활성화 출력의 히스토그램을 기록
                # embeddings_freq=1 : 1에포크 마다 임베딩 데이터 기록
                # profile_batch : https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard
               profile_batch=100000000),
    EarlyStopping(monitor='val_loss',patience=3)
]
model.fit(Xtrain, ytrain, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, validation_data=(Xtest, ytest), callbacks=callbacks_list)

C:\Users\TJ\anaconda3\lib\site-packages\keras\callbacks\tensorboard_v2.py:102: UserWarning: The TensorBoard callback does not support embeddings display when using TensorFlow 2.0. Embeddings-related arguments are ignored.
  warnings.warn('The TensorBoard callback does not support '


Train on 5668 samples, validate on 1418 samples
Epoch 1/100
5668/5668 [==============================] - 2s 378us/step - loss: 0.6522 - acc: 0.6316 - val_loss: 0.6084 - val_acc: 0.6798
Epoch 2/100
5668/5668 [==============================] - 1s 231us/step - loss: 0.5700 - acc: 0.7378 - val_loss: 0.5107 - val_acc: 0.7581
Epoch 3/100
5668/5668 [==============================] - 1s 210us/step - loss: 0.4663 - acc: 0.8126 - val_loss: 0.4022 - val_acc: 0.8547
Epoch 4/100
5668/5668 [==============================] - 1s 200us/step - loss: 0.3917 - acc: 0.8439 - val_loss: 0.3493 - val_acc: 0.8526
Epoch 5/100
5668/5668 [==============================] - 1s 191us/step - loss: 0.3457 - acc: 0.8634 - val_loss: 0.3053 - val_acc: 0.8731
Epoch 6/100
5668/5668 [==============================] - 1s 193us/step - loss: 0.3043 - acc: 0.8829 - val_loss: 0.2767 - val_acc: 0.8858
Epoch 7/100
5668/5668 [==============================] - 1s 205us/step - loss: 0.2815 - acc: 0.8848 - val_loss: 0.2447 - val_acc: 

In [32]:
loss_test, acc_test = model.evaluate(Xtest, ytest, batch_size=BATCH_SIZE)
print("Test loss: %.3f, accuracy: %.3f" % (loss_test, acc_test))

1418/1418 [==============================] - 0s 70us/step
Test loss: 0.098, accuracy: 0.961


In [33]:
for i in range(5):
    idx = np.random.randint(len(Xtest))
    xtest = Xtest[idx].reshape(1,40)
    ylabel = ytest[idx]
    ypred = model.predict(xtest)[0][0]
    sent = " ".join([index2word[x] for x in xtest[0].tolist() if x != 0])
    print("%.0f\t%d\t%s" % (ypred, ylabel, sent))

0	0	so brokeback mountain was really depressing .
0	0	my dad 's being stupid about brokeback mountain ...
0	0	oh , and brokeback mountain was a terrible movie .
0	0	as i sit here , watching the mtv movie awards , i am reminded of how much i despised the movie brokeback mountain .
0	0	as i sit here , watching the mtv movie awards , i am reminded of how much i despised the movie brokeback mountain .
